Step 1: build a surface water network. You can "pickle" this, so it doesn't need to be repeated.

n = swn.SurfaceWaterNetwork.from_lines(gdf.geometry)

n.to_pickle("surface-water-network.pkl")

# then in a later session, skip the above and just do:

n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

Step 2: load a MF6 model, then find the intersections:

sim = flopy.mf6.MFSimulation.load(...)

m = sim.get_model(...)

nm = swn.SwnMf6.from_swn_flopy(n, m)

Most of the results will be in the nm.reaches property, but other reach datasets will need to be specified, including "man", "rbth", "rgrd", "rhk", "rtp", and "rwid". There are two methods to format the PACKAGEDATA:

nm.flopy_packagedata

nm.write_packagedata("packagedata.dat")

Similar with CONNECTIONDATA:

nm.flopy_connectiondata

nm.write_connectiondata("connectiondata.dat")

There are a few "helper" methods to sort out things like "set_reach_slope" based on a few methods. One missing one is "set_reach_elevation" or whatever to make the reaches fit in the layer and/or move the layer elevations to fit the stream.

And lastly, there is no PERIOD data yet. I'm working on this, which is holding up the merge.


In [1]:
import geopandas
import os
import swn
import flopy
import numpy as np
import time

# SW network pickle

#only do this once... takes forever... then load pickle
gdb_dir = 'D:\modelling\data'
gdb_fname = 'nzRec2_v5.gdb'
gdb_path = os.path.join(gdb_dir, gdb_fname)
# Read national data of streams
gdf_lines = geopandas.read_file(gdb_path, layer='riverlines')
gdf_lines.set_index('nzsegment', inplace=True, verify_integrity=True)
gdf_ws = geopandas.read_file(gdb_path, layer='rec2ws')
gdf_ws.set_index('nzsegment', inplace=True, verify_integrity=True)
# Convert MultiLineString -> LineString
lines = gdf_lines.geometry.apply(lambda x: x.geoms[0]) #what is geoms[0]
polygons = gdf_ws.geometry.apply(lambda x: x.geoms[0])
#ni_lines = gdf_lines.loc[gdf_lines.index < 10000000, "geometry"]
# requires reindex otherwise failure in core.from_lines
t0=time.time()
n = swn.SurfaceWaterNetwork.from_lines(lines,polygons.reindex(index=lines.index))
print(time.time()-t0)
n.to_pickle("surface-water-network.pkl")

# get the pickle

In [2]:
n = swn.SurfaceWaterNetwork.from_pickle("surface-water-network.pkl")

# Load MF6 model

In [3]:
os.getcwd()

'D:\\modelling\\surface-water-network'

In [4]:
sim_ws=os.path.join('..','zmodels','20210622_simulation','wairau_240_3')
model_name='wairau_240_3'
sim=flopy.mf6.MFSimulation.load(sim_ws=sim_ws)
gwf=sim.get_model(model_name)

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package ghb...
    loading package rch...
    loading package drn...
    loading package npf...
    loading package sto...
    loading package oc...
  loading ims package wairau_240_3...


## spatial reference for model

In [5]:
#sr=flopy.utils.reference.SpatialReference.from_gridspec(os.path.join(sim_ws,model_name+'.grid.spc'))
#gwf.dis.xorigin=sr.xul
#gwf.dis.yorigin=sr.yul-np.sum(gwf.dis.delr.data)
#gwf.dis.write()

In [6]:
type(gwf)

flopy.mf6.mfmodel.MFModel

In [7]:
# this also takes forever
t0=time.time()
ngwf = swn.SwnMf6.from_swn_flopy(n, gwf)
print(time.time()-t0)

15:48:28.584:WARNING:SwnMf6:failed to merge segnum 11018756 at (36, 456): MULTILINESTRING ((1686011.163437578 5419920, 1686005.0497 5419913.8872, 1685920 5419913.893816454, 1685915.0698 5419913.8942), (1686200.0061 5419913.872, 1686160 5419913.875112284, 1686155.0162 5419913.8755, 1686148.892659643 5419920))
15:48:33.826:WARNING:SwnMf6:failed to merge segnum 11020068 at (50, 455): MULTILINESTRING ((1685680 5416399.543781261, 1685759.530389482 5416320, 1685764.8128 5416314.7167, 1685794.8061 5416314.7142), (1685689.8462 5416509.6788, 1685680 5416499.834263151))
15:48:38.701:WARNING:SwnMf6:failed to merge segnum 11021078 at (61, 424): MULTILINESTRING ((1678221.3706 5413855.9237, 1678240 5413855.922126469, 1678266.3597 5413855.9199, 1678296.355 5413885.9102, 1678326.3478 5413885.9077, 1678360.445840736 5413920), (1678412.222198316 5413920, 1678416.3286 5413915.8929))
15:48:40.060:WARNING:SwnMf6:failed to merge segnum 11020969 at (62, 409): MULTILINESTRING ((1674693.515613634 5413440, 1674

15:50:17.684:WARNING:SwnMf6:failed to merge segnum 11028594 at (145, 266): MULTILINESTRING ((1640414.9104 5393494.9469, 1640414.912822535 5393520, 1640414.9162 5393554.9287, 1640320 5393649.863574369), (1640320 5393759.910376494, 1640320.089641139 5393760))
15:50:20.059:WARNING:SwnMf6:failed to merge segnum 11028827 at (147, 376): MULTILINESTRING ((1666942.2461 5393117.3249, 1666960 5393135.074972427), (1666960 5393192.302706609, 1666957.2502 5393192.303, 1666957.259751797 5393280, 1666957.26 5393282.2788))
15:50:20.414:WARNING:SwnMf6:failed to merge segnum 11029270 at (146, 460): MULTILINESTRING ((1686932.148620711 5393280, 1686932.1523 5393309.9678, 1686880 5393362.132881958), (1686880 5393407.773780726, 1686992.253762132 5393520, 1687022.16 5393549.8989))
15:50:21.743:WARNING:SwnMf6:failed to merge segnum 11029467 at (148, 225): MULTILINESTRING ((1630720 5392993.914177159, 1630673.923127443 5393040, 1630667.8867 5393046.0376, 1630667.8896 5393076.0282), (1630712.8506 5392821.1042, 1

15:51:37.838:WARNING:SwnMf6:failed to merge segnum 11035340 at (204, 179): MULTILINESTRING ((1619660.0048 5379341.5929, 1619660.006825443 5379360, 1619660.0114 5379401.5733, 1619680 5379421.557501541), (1619680 5379531.547604204, 1619660.0279 5379551.5241))
15:51:41.700:WARNING:SwnMf6:failed to merge segnum 11035803 at (207, 282): MULTILINESTRING ((1644231.208964622 5378640, 1644162.1861 5378709.0389), (1644432.097 5378649.0258, 1644402.1061 5378649.0293, 1644400 5378646.923684495, 1644393.074722389 5378640))
15:51:46.927:WARNING:SwnMf6:failed to merge segnum 11036176 at (210, 168): MULTILINESTRING ((1616973.088640042 5377920, 1616960.7361 5377932.3553, 1616900.7559 5377932.362, 1616900.7659 5378022.3323), (1617050.7064 5377932.3453, 1617040 5377921.64132731, 1617038.358300491 5377920))
15:51:50.103:WARNING:SwnMf6:failed to merge segnum 11036094 at (210, 334): MULTILINESTRING ((1656880 5378046.045915339, 1656788.3601 5378137.7082, 1656788.362812956 5378160, 1656788.3674 5378197.6908), 

In [8]:
# https://modflow6.readthedocs.io/en/latest/_mf6io/gwf-sfr.html?highlight=ustrf#block-packagedata
# started from sagehen example, tweaked
# can do ngwf.default_packagedata() now?
d={'rwid':10.0,'man':0.04,'ustrf':1.0,'ndv': 0}
for k in ["man", "rwid"]:
    ngwf.reaches[k]=d[k]

# was taking much time to fail with no zcoord, now takes 1.2 sec
t0=time.time()
#zcoord_ab or grid_top
try:
    ngwf.set_reach_slope(method='zcoord_ab')
except:
    print(time.time()-t0)


In [9]:
# check for nans, should have been fixed, taken care of
ngwf.set_reach_slope(method='top_len')

15:54:30.087:WARNING:SwnMf6:enforcing min_slope for 208 reaches (0.62%)


In [25]:
# no rno=0 so missing labels error, floats because of nans
top=gwf.dis.top.array
botm=gwf.dis.botm.array

In [ ]:
drbth=2.0 #default rbth
dbit=0.1
cont=True
while cont:
    chg=0
    for idx,r in ngwf.reaches.iterrows():
        if r['to_rno']!=0:
            i=int(r['i'])
            j=int(r['j'])
            # get list of i,j for upstreams too?
            trno=ngwf.reaches.loc[idx,'to_rno']
            ti=int(ngwf.reaches.loc[trno,'i'])
            tj=int(ngwf.reaches.loc[trno,'j']) 
            if top[i,j]<=top[ti,tj]:
                mtop=np.mean([top[i,j],top[ti,tj]])
                dz=top[ti,tj]-mtop+dbit
                top[i,j]=mtop+dbit
                top[ti,tj]=mtop-dbit
                for b in range(0,botm.shape[0]):
                    botm[b][ti,tj]=botm[b][ti,tj]-dz
                chg=chg+1
            ngwf.reaches.loc[idx,'rtp']=top[i,j]
            ngwf.reaches.loc[idx,'rbth']=drbth
            ngwf.reaches.loc[trno,'rtp']=top[ti,tj]
            ngwf.reaches.loc[trno,'rbth']=drbth

        if (ngwf.reaches.loc[trno,'rtp']-ngwf.reaches.loc[trno,'rbth'])<botm[0][ti,tj]:
            print('derp, your maths suck')
    if chg==0:
        cont=False
    else:
        print('{} changed'.format(chg))

11077 changed
8572 changed
8398 changed
8305 changed
8322 changed
8283 changed
8332 changed
8280 changed
8329 changed
8322 changed
8359 changed
8331 changed
8421 changed
8410 changed
8473 changed
8412 changed
8491 changed
8443 changed
8490 changed
8487 changed
8560 changed


In [ ]:
f=gwf.dis.top.get_file_entry().split("'")[1]
fpth=os.path.join(sim_ws+'_sfr',f)
np.savetxt(f,top)

In [ ]:
for b in range(0,botm.shape[0]):
    f=gwf.dis.botm[b].fname
    fpth=os.path.join(sim_ws+'_sfr',f)
    np.savetxt(f,botm[b])

In [ ]:
ngwf.set_reach_data_from_array('rhk',gwf.npf.k.array[0])

In [ ]:
n.segments.to_csv(os.path.join(sim_ws+'_sfr',model_name+'_znseg_mf6rch.csv'))

In [ ]:
mask=[s for s in n.segments.index if (len(n.segments.loc[s,'from_segnums'])==0) & (n.segments.loc[s,'to_segnum']==0)]

In [ ]:
len(mask)

In [ ]:
n.segments.drop(mask,inplace=True)

In [ ]:


# There are two methods to format the PACKAGEDATA:
ngwf.flopy_packagedata
ngwf.write_packagedata(os.path.join(sim_ws+'_sfr',model_name+'.sfr0.reach.dat'))

#Similar with CONNECTIONDATA:
ngwf.flopy_connectiondata
ngwf.write_connectiondata(os.path.join(sim_ws+'_sfr',model_name+'.sfr0.connection.dat'))


In [ ]:
sfr=flopy.mf6.ModflowGwfsfr(gwf,packagedata={'filename':model_name+'.sfr0.reach.dat'},
                            connectiondata={'filename':model_name+'.sfr0.connection.dat'},
                           nreaches=len(ngwf.reaches),budget_filerecord=model_name + "_sfr.bud",
                            save_flows=True)
#gwf.register_package(sfr)
sfr.write()

In [ ]:
model_name

In [ ]:
gwf.write()

In [ ]:
help(ngwf)

# write shapefile, but not sfr info from grid intersection

In [ ]:
swn.file.gdf_to_shapefile(n.segments, 'segments.shp')